In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=a6dec1b38085edde8e887f078ad51402eb7021577aab6c2406c4a681d7a02f92
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [2]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import glob
from pathlib import Path

**SKELETON VIDEO TO REALISTIC VIDEO**

In [3]:
# Install required package
!pip uninstall dominate
!pip install dominate --quiet

In [4]:
%cd /content/

/content


In [5]:
!unzip /content/drive/Shareddrives/298A/trial/pix2pixHD.zip

Archive:  /content/drive/Shareddrives/298A/trial/pix2pixHD.zip
   creating: pix2pixHD/
  inflating: __MACOSX/._pix2pixHD    
  inflating: pix2pixHD/encode_features.py  
  inflating: __MACOSX/pix2pixHD/._encode_features.py  
   creating: pix2pixHD/options/
  inflating: __MACOSX/pix2pixHD/._options  
  inflating: pix2pixHD/.DS_Store     
  inflating: __MACOSX/pix2pixHD/._.DS_Store  
   creating: pix2pixHD/util/
  inflating: __MACOSX/pix2pixHD/._util  
  inflating: pix2pixHD/precompute_feature_maps.py  
  inflating: __MACOSX/pix2pixHD/._precompute_feature_maps.py  
   creating: pix2pixHD/datasets/
  inflating: __MACOSX/pix2pixHD/._datasets  
   creating: pix2pixHD/checkpoints/
  inflating: __MACOSX/pix2pixHD/._checkpoints  
  inflating: pix2pixHD/run_engine.py  
  inflating: __MACOSX/pix2pixHD/._run_engine.py  
   creating: pix2pixHD/models/
  inflating: __MACOSX/pix2pixHD/._models  
  inflating: pix2pixHD/test.py       
  inflating: __MACOSX/pix2pixHD/._test.py  
  inflating: pix2pixHD/R

Reading the video and breaking into frames

In [ ]:
import cv2
import os, shutil
# Open the video file
merged_output = '/content/drive/Shareddrives/298A/lookup_gloss_videolist/DO_YOU_THINK_WE_CAN_AVOID_ACCIDENT_?.mp4'
# Open the video file
video = cv2.VideoCapture(merged_output)
path = "/content/drive/Shareddrives/298A/frames/initial"

# Delete any previous frames in the folder
for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Check if the video file was opened successfully
if not video.isOpened():
    print("Error opening video file")
    exit()

# Set the frame number to start with
frame_num = 0

# Loop through the video frames
while True:
    # Read the next frame from the video
    ret, frame = video.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Save the frame as an image file
    # cv2.imwrite(f"frame{frame_num}.png", frame)
    cv2.imwrite(os.path.join(path , f"frame{frame_num}.png"), frame)

    # Increment the frame number
    frame_num += 1

# Release the video object
video.release()

Cropping the inferred image

In [ ]:
import os
import cv2

# Set the paths for the input folder containing images and the output folder
input_folder = '/content/drive/Shareddrives/298A/frames/initial'
output_folder = '/content/drive/Shareddrives/298A/frames/test_A'

# Delete any previous frames in the folder

for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Desired crop dimensions and offset
crop_width = 400
crop_height = 336
offset = 40

# Loop over the images in the input folder
for filename in os.listdir(input_folder):
    # Read the image
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)
    # Calculate the crop coordinates
    image_height, image_width = img.shape[:2]
    start_x = max(0, int((image_width - crop_width) / 2))
    start_y = max(0, int((image_height - crop_height) / 2)) - offset
    
    # Perform the crop
    cropped_image = img[start_y:start_y + crop_height, start_x:start_x + crop_width]
    
    # Create the output path for the cropped image
    output_path = os.path.join(output_folder, filename)
    
    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

In [ ]:
%cd /content/pix2pixHD

/content/pix2pixHD


Running the GAN inference on the above frames

In [ ]:
# Conduct inference on test images. Inference results will be stored in "results" directory
!python3 test.py --name pose2sign --dataroot '/content/drive/Shareddrives/298A/frames' --label_nc 0 --no_instance --resize_or_crop None --how_many 100

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /content/drive/Shareddrives/298A/frames
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 100
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: pose2sign
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: None
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset 

To sort the files in alphanumeric order

In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
from os.path import isfile, join

pathIn = '/content/pix2pixHD/results/pose2sign/test_latest/images'
pathOut = 'GAN_generated_new.mp4'
fps = 10
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()

In [ ]:
sorted_list = sorted_alphanumeric(files)

Creating the video from inferred frames

In [ ]:
import cv2
import fnmatch

for file in sorted_list:
  if fnmatch.fnmatch(file, '*_synthesized_image.jpg'):
    filename = pathIn +'/'+ file
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

Compressing the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/pix2pixHD/GAN_generated_new.mp4"

# Compressed video path
compressed_path = "/content/pix2pixHD/compressed_generated_new.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

0

Displayin ouput video

In [ ]:
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=700 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)